Prototyping notebook for predicting stock volaitility, prices, etc using extra data from web trends, news, etc. 

1) Download and extract https://www.kaggle.com/datasets/footballjoe789/us-stock-dataset

2) python packages needed: compress_pickle[lz4] pandas numpy yahooquery seaborn

2) Copy "Stock_List.csv" and "Stocks/*" to "data/". You can delete the other files as they are not used (at least yet).

In [ ]:
# %pip install compress_pickle[lz4] pandas numpy yahooquery seaborn

In [ ]:
import pandas as pd
import numpy as np
import seaborn
from glob import glob
from compress_pickle import dump, load
import os
from yahooquery import Ticker
import timeit
import time
import datetime
from get_stock_search_terms import aquire_stock_search_terms as aquire_terms

In [ ]:
# Gather the company info for all the ticker symbols and return a dataframe with relevant search terms for each company.
search_terms = aquire_terms('data/Stocks/')
search_terms.data

Datasets:
We need a historical dataset of stock prices.
Secondly, we need a dataset of web trends for the same time period. 

On the other hand, once the model is trained, it needs to be able to aquire live data that looks the same as the training data.

Possible sources:
   - twitter
   - google trends

Possible models:
    - Train a model to process text into an encoding
    - Train a model to do stock price prediction
    - Combine the two models to predict stock price/volatility (not sure which is best) from text with greater accuracy